In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ tệp CSV
url = "./BTL/Data/breast-cancer.csv"
data = pd.read_csv(url)

data['diagnosis'] = data['diagnosis'].replace({'M': 1, 'B': 0})
# Lấy cột cần di chuyển
column_to_move = data['diagnosis']

# Loại bỏ cột này khỏi DataFrame
data.drop(columns=['diagnosis'], inplace=True)

# Thêm cột này lại vào cuối DataFrame
data['diagnosis'] = column_to_move

# Xác định đặc trưng và biến mục tiêu
X = data.drop(columns=["diagnosis"])  # Thay "target_column" bằng tên cột mục tiêu của bạn
Y = data["diagnosis"]
data.head(10)


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1
5,843786,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,1
6,844359,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,1
7,84458202,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,1
8,844981,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,1
9,84501001,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,1


In [2]:

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(Y_test)


204    0
70     1
131    1
431    0
540    0
      ..
486    0
75     1
249    0
238    0
265    1
Name: diagnosis, Length: 114, dtype: int64


In [3]:
# Triển khai Random Forest từ đầu
from sklearn.tree import DecisionTreeClassifier

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.models = []

    def fit(self, X, Y):
        np.random.seed(self.random_state)
        for _ in range(self.n_estimators):
            sample_indices = np.random.choice(X.shape[0], size=X.shape[0], replace=True)
            X_sampled = X.iloc[sample_indices]
            Y_sampled = Y.iloc[sample_indices]

            model = self.create_decision_tree(X_sampled, Y_sampled, self.max_depth)
            self.models.append(model)

    def create_decision_tree(self, X, Y, max_depth):
        model = DecisionTreeClassifier(max_depth=max_depth)
        model.fit(X, Y)
        return model

    def predict(self, X):
        predictions = [model.predict(X) for model in self.models]
        # Kết quả dự đoán từ các cây quyết định được tổng hợp ở đây, có thể cần áp dụng ngưỡng (threshold) để chuyển thành nhãn nhị phân
        # Ví dụ: 0 nếu giá trị < 0.5, 1 nếu giá trị >= 0.5
        predictions = [1 if p >= 0.5 else 0 for p in np.mean(predictions, axis=0)]
        return predictions

In [4]:

# Tạo và huấn luyện mô hình Random Forest
clf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
clf.fit(X_train, Y_train)




In [5]:
# Dự đoán trên tập kiểm tra
Y_pred = clf.predict(X_test)
print(Y_test)

204    0
70     1
131    1
431    0
540    0
      ..
486    0
75     1
249    0
238    0
265    1
Name: diagnosis, Length: 114, dtype: int64


In [6]:
# Đánh giá mô hình
from sklearn.metrics import accuracy_score



In [7]:
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Độ chính xác: {accuracy * 100:.2f}%")

Độ chính xác: 95.61%


In [8]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, Y_train)
model.score(X_test,Y_test)


0.9649122807017544

In [9]:
model = RandomForestClassifier(n_estimators=40)
model.fit(X_train, Y_train)
model.score(X_test,Y_test)

0.9649122807017544